<a href="https://colab.research.google.com/github/anas1IA/MLP-Hyperparameters-optimization/blob/main/mlp_ga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import libraries

In [238]:
import numpy as np
import pandas as pd
import pygad
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as  f
from sklearn.model_selection  import train_test_split

### import data (Iris.csv)

In [239]:
data = pd.read_csv("Iris.csv")
data.tail()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica
149,150,5.9,3.0,5.1,1.8,Iris-virginica


In [240]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [241]:
data["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [242]:
data.shape

(150, 6)

## Do some trnsformation for data

In [243]:
data = data.drop(columns= "Id")

In [244]:
data['Species'] = data['Species'].replace('Iris-setosa',0.0)
data['Species'] = data['Species'].replace('Iris-versicolor',1.0)
data['Species'] = data['Species'].replace('Iris-virginica',2.0)

In [245]:
data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [246]:
X = data.drop('Species',axis=1)
y = data['Species']
X

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [247]:
# convert thesse to numpy array
X = X.values
y = y.values

In [248]:
X.dtype

dtype('float64')

In [249]:
X ,y

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

In [250]:
X_train ,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
y_train

array([1., 1., 2., 1., 1., 1., 0., 2., 0., 0., 0., 2., 1., 1., 0., 1., 2.,
       1., 0., 2., 2., 2., 0., 0., 2., 0., 1., 1., 0., 1., 0., 0., 0., 2.,
       1., 2., 1., 2., 0., 1., 1., 2., 1., 0., 1., 2., 2., 0., 0., 2., 1.,
       0., 1., 0., 0., 2., 1., 0., 1., 0., 2., 2., 1., 2., 2., 1., 1., 2.,
       0., 1., 1., 0., 1., 2., 2., 0., 0., 1., 1., 1., 0., 1., 1., 2., 1.,
       0., 2., 0., 0., 0., 1., 2., 2., 2., 1., 0., 1., 2., 0., 0., 0., 0.,
       0., 2., 2., 0., 1., 2., 1., 2., 1., 0., 2., 0., 2., 2., 1., 2., 1.,
       0.])

In [251]:
#convert features to tensors
X_train = torch.FloatTensor(X_train)
X_test= torch.FloatTensor(X_test)
X_train

tensor([[5.6000, 3.0000, 4.1000, 1.3000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [6.4000, 3.1000, 5.5000, 1.8000],
        [6.5000, 2.8000, 4.6000, 1.5000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [5.5000, 2.5000, 4.0000, 1.3000],
        [5.0000, 3.2000, 1.2000, 0.2000],
        [6.7000, 2.5000, 5.8000, 1.8000],
        [4.8000, 3.4000, 1.9000, 0.2000],
        [4.6000, 3.4000, 1.4000, 0.3000],
        [4.8000, 3.4000, 1.6000, 0.2000],
        [6.1000, 3.0000, 4.9000, 1.8000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [5.2000, 2.7000, 3.9000, 1.4000],
        [5.8000, 4.0000, 1.2000, 0.2000],
        [5.7000, 2.6000, 3.5000, 1.0000],
        [6.5000, 3.2000, 5.1000, 2.0000],
        [6.1000, 2.8000, 4.7000, 1.2000],
        [5.2000, 4.1000, 1.5000, 0.1000],
        [7.2000, 3.2000, 6.0000, 1.8000],
        [6.3000, 2.5000, 5.0000, 1.9000],
        [7.7000, 2.6000, 6.9000, 2.3000],
        [5.0000, 3.4000, 1.5000, 0.2000],
        [4.4000, 3.0000, 1.3000, 0

In [252]:
# convert labels to long values
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
y_train

tensor([1, 1, 2, 1, 1, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0, 1, 2, 1, 0, 2, 2, 2, 0, 0,
        2, 0, 1, 1, 0, 1, 0, 0, 0, 2, 1, 2, 1, 2, 0, 1, 1, 2, 1, 0, 1, 2, 2, 0,
        0, 2, 1, 0, 1, 0, 0, 2, 1, 0, 1, 0, 2, 2, 1, 2, 2, 1, 1, 2, 0, 1, 1, 0,
        1, 2, 2, 0, 0, 1, 1, 1, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 1, 0,
        1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2, 1, 2, 1, 0])

## Import library has related to genetic algorithm

In [253]:
!pip install torch pygad

In [255]:
class MyDataset(Dataset):
    def __init__(self, Tarin_data ,labels):
        #self.X = torch.rand(size, 2)
        self.X = Tarin_data
        #self.y = torch.matmul(self.X, torch.Tensor([1, 2]).t())
        self.y = labels

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [256]:
class IsNeuronAct(nn.Module):

  def __init__(self, mask: torch.Tensor):
      super(IsNeuronAct, self).__init__()
      self.mask = nn.Parameter(torch.Tensor(mask), requires_grad=False)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return x * self.mask

In [257]:
class MLP(nn.Module):

  def __init__(self, mask: torch.Tensor):
    super(MLP, self).__init__()

    self.l1 = nn.Linear(4, 8)
    self.act1 = IsNeuronAct(mask)
    self.re = nn.ReLU()
    self.l2 = nn.Linear(8, 3)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.l1(x)
    #x =self.re(x)
    x = self.act1(x)
    return self.l2(x)

In [258]:
dataset = MyDataset(X_train ,y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [259]:
for inputs, targets in dataloader:
  print(inputs ,targets)


tensor([[6.6000, 2.9000, 4.6000, 1.3000],
        [4.6000, 3.4000, 1.4000, 0.3000],
        [5.3000, 3.7000, 1.5000, 0.2000],
        [5.2000, 4.1000, 1.5000, 0.1000],
        [6.7000, 3.1000, 4.4000, 1.4000],
        [4.8000, 3.4000, 1.9000, 0.2000],
        [6.3000, 2.7000, 4.9000, 1.8000],
        [7.3000, 2.9000, 6.3000, 1.8000],
        [6.3000, 2.8000, 5.1000, 1.5000],
        [6.1000, 2.8000, 4.0000, 1.3000],
        [6.7000, 3.0000, 5.2000, 2.3000],
        [6.3000, 3.3000, 6.0000, 2.5000],
        [5.1000, 3.8000, 1.9000, 0.4000],
        [5.1000, 3.8000, 1.6000, 0.2000],
        [7.4000, 2.8000, 6.1000, 1.9000],
        [7.2000, 3.2000, 6.0000, 1.8000],
        [4.5000, 2.3000, 1.3000, 0.3000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [4.4000, 2.9000, 1.4000, 0.2000],
        [6.3000, 3.4000, 5.6000, 2.4000],
        [4.8000, 3.1000, 1.6000, 0.2000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [6.0000, 2.9000, 4.5000, 1.5000],
        [5.4000, 3.9000, 1.3000, 0

In [260]:
results: list[tuple[nn.Module, float]] = []

def train_model(mask, num_epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = MLP(mask)
    model.to(device)

    #mse = nn.MSELoss()
    critersion = nn.CrossEntropyLoss()
    opt = torch.optim.Adam(model.parameters(),lr = 0.01)
    #opt = torch.optim.SGD(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            loss =  critersion(outputs, targets)

            opt.zero_grad()
            loss.backward()
            opt.step()

    print(f"loss: {loss.item()}")
    results.append((model, loss.item()))
    return loss

In [269]:
#train_model(torch.Tensor([1.0, 1.0, 1.0, 1.0]))
train_model(torch.Tensor([ 0, 1, 1, 1, 1,1,1,1]))


loss: 0.07064714282751083


tensor(0.0706, device='cuda:0', grad_fn=<NllLossBackward0>)

In [270]:
def fitness(ga_instance, solution, solution_idx):
    loss = train_model(solution)

    return 1/(float(loss)+ 1e-17) + 8 - float(np.sum(solution))

In [271]:
num_genes = 8 # # Définir le nombre de gènes dans chaque individu de la population (num_genes = len(mask))

num_generations = 15# Définir le nombre de générations pour exécuter l'algorithme génétique

population_size = 5# number of individules(soultions in this case solution weights of module )

initial_population = np.random.choice([0, 1], size=(population_size, num_genes))# Générer une population initiale aléatoire avec des gènes codés en 0 ou 1

In [274]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=2,
    initial_population=initial_population.copy(),
    fitness_func=fitness,
    mutation_type="random",
    crossover_probability=0.8,
    mutation_probability=0.2,
    parent_selection_type="rank",
    crossover_type="single_point",
    keep_parents=2,
    gene_type=int,
)

In [275]:
ga_instance.run()

loss: 0.028529776260256767
loss: 0.09133604168891907
loss: 0.04364674910902977
loss: 0.08864898234605789
loss: 0.03444841876626015
loss: 0.09365943819284439
loss: 0.05044605955481529
loss: 0.0432913601398468
loss: 0.024907564744353294
loss: 0.09029433131217957
loss: 0.1064978837966919
loss: 0.07511337846517563
loss: 0.05515240505337715
loss: 0.04962683096528053
loss: 0.0877346396446228
loss: 0.11441423743963242
loss: 0.11808302998542786
loss: 0.02654140628874302
loss: 0.07254064083099365
loss: 0.031189702451229095
loss: 0.021209731698036194
loss: 0.06537306308746338
loss: 0.05978820100426674
loss: 0.05939740315079689
loss: 0.03008895367383957
loss: 0.016223108395934105
loss: 0.06422676891088486
loss: 0.08379138261079788
loss: 0.06683953106403351
loss: 0.07033618539571762
loss: 0.12356331199407578
loss: 0.07729055732488632
loss: 0.018849140033125877
loss: 0.08404151350259781
loss: 0.05461880564689636
loss: 0.04030327871441841


In [276]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()

print("Best solution:", solution)
print("Fitness value:", solution_fitness)

loss: 0.037177592515945435
Best solution: [0 1 1 0 1 0 0 0]
Fitness value: 66.64046837353457


In [290]:
# so in this case number of neurons optimal that we can use  and obtain good result is
num_neurons = sum(solution)
num_neurons

3